In [1]:
import numpy as np
import matplotlib.pyplot as plt
import keras
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchdiffeq
import tqdm

Using TensorFlow backend.


In [2]:
(X_train,Y_train),(X_test,Y_test) = keras.datasets.mnist.load_data()
X_train,X_test = X_train[:,np.newaxis]/255,X_test[:,np.newaxis]/255

In [3]:
class ODEFunc(nn.Module):
    
    def __init__(self, dim):
        super().__init__()
        self.conv1 = nn.Conv2d(dim+1,dim,3,padding=1)
        self.conv2 = nn.Conv2d(dim,dim,3,padding=1)
    
    def forward(self, t, x):
        t = t.expand(x.size()[:1] + (1,) + x.size()[2:])
        x = torch.cat([x,t], dim=1)
        x = self.conv1(x)
        x = torch.tanh(x)
        x = self.conv2(x)
        x = torch.tanh(x)
        return x

In [4]:
class ODEBlock(nn.Module):
    
    def __init__(self, dim):
        super().__init__()
        self.func = ODEFunc(dim)
        self.itime = torch.Tensor([0,1])
    
    def forward(self, x):
        itime = self.itime.type_as(x)
        return torchdiffeq.odeint_adjoint(
            self.func, x, itime, rtol=1e-2, atol=1e-2)[1]

In [5]:
class Net(nn.Module):
    
    def __init__(self):
        super().__init__()
        self.norm0 = nn.BatchNorm2d(1)
        self.conv1 = nn.Conv2d(1,8,7,padding=0)
        self.norm1 = nn.BatchNorm2d(8)
        self.conv2 = nn.Conv2d(8,64,7,padding=0)
        self.norm2 = nn.BatchNorm2d(64)
        self.conv3 = ODEBlock(64)
        self.norm3 = nn.BatchNorm2d(64)
        self.conv4 = ODEBlock(64)
        self.norm4 = nn.BatchNorm2d(64)
        self.conv5 = ODEBlock(64)
        self.norm5 = nn.BatchNorm2d(64)
        self.fc = nn.Linear(64,10)
    
    def forward(self, x):
        x = self.norm0(x)
        x = self.conv1(x)
        x = self.norm1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = self.norm2(x)
        x = F.relu(x)
        x = F.max_pool2d(x,2)
        x = self.conv3(x)
        x = self.norm3(x)
        x = F.relu(x)
        x = F.max_pool2d(x,2)
        x = self.conv4(x)
        x = self.norm4(x)
        x = F.relu(x)
        x = F.max_pool2d(x,2)
        x = self.conv5(x)
        x = self.norm5(x)
        x = F.relu(x)
        x = F.max_pool2d(x,2)
        x = x.view((-1,64))
        x = self.fc(x)
        x = F.softmax(x, dim=-1)
        return x

In [6]:
device = torch.device('cuda')
data,test,label = X_train,X_test,Y_train
data = torch.Tensor(data).to(device)
test = torch.Tensor(test).to(device)
label = torch.Tensor(label).to(device).type(torch.long)
net = Net().to(device)
los = nn.CrossEntropyLoss().to(device)
opt = torch.optim.Adam(net.parameters())

In [7]:
net = net.train()
for _ in range(3):
    loss_avg = 0
    loss_cnt = 0
    for batch_idx in tqdm.tqdm_notebook(range(0,data.shape[0],128)):
        batch = data[batch_idx:batch_idx+128]
        opt.zero_grad()
        output = net(batch)
        loss = los(output, label[batch_idx:batch_idx+128])
        loss.backward()
        opt.step()
        loss_avg += loss.item()
        loss_cnt += 1
    print(loss_avg/loss_cnt)


1.5517927232835846



1.4810676833968173



1.4768741822191902


In [8]:
net = net.eval()
test_hit = 0
for batch_idx in tqdm.tqdm_notebook(range(0,test.shape[0],128)):
    batch = test[batch_idx:batch_idx+128]
    pred = net(batch)
    pred = torch.argmax(pred, dim=-1)
    pred = pred.cpu().detach().numpy()
    test_hit += np.sum(pred == Y_test[batch_idx:batch_idx+128])
test_hit/Y_test.shape[0]

0.9857